<a href="https://colab.research.google.com/github/annaliese-estes/quantum-8-ball/blob/main/Quantum_Masterclass_Answer_Key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Magic 8 Ball

Instructor:  
Annaliese Estes  

# Magic 8 Ball

Here, we will build a Magic 8 Ball program that randomly returns 1 of 8 possible responses. How do we use quantum computing to generate a random number? Computational space scales exponentially in quantum computing. Thus, if we need a random result out of 8 possibilities, our program needs to take a measurement of a quantum state vector that consists of 8 basis states, which represent the computational space of 3 qubits in an equal superposition.



In [ ]:
# install Qiskit with visualization

!pip install qiskit[visualization]
!pip install qiskit-ibm-runtime

In [ ]:
# install additional packages

from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import math
import matplotlib.pyplot as plt

In [ ]:
## Save your IBM Quantum account and set it as your default account.
QiskitRuntimeService.save_account(

    token="YourTokenHere",

    instance="YourInstanceHere", # Optionally specify the instance to use.

    set_as_default=True,

    # Use `overwrite=True` if you're updating your token.
    #overwrite=True,
)

# Load saved credentials
service = QiskitRuntimeService()

In [ ]:
# define Magic 8 Ball responses

responses = ["Yes", "Not today", "Definitely", "Try again", "Signs point to yes", "Not likely", "Sure thing!", "Outlook not so good"]

## Qiskit Patterns Step 1: Map problem to quantum circuits

We can think of this step as mapping our problem to be run on a quantum computer. This step needs to be done for any quantum computation, because our instinct is to think of problems in a classical way, while quantum computers work differently.

Problem:
I need a program to generate a random number out of 8.

Mapping to a classical computer: generate a random integer in range(1,9)

Mapping to a quantum computer: put 3 qubits into an equal superposition, which creates a quantum state vector with 8 basis states, each with an equal probability of being the result of a measurement

In [ ]:
# set up a Quantum circuit with 3 qubits
qc = QuantumCircuit(3)

# place a Hadamard gate on qubits 0, 1, and 2
qc.h(0)
qc.h(1)
qc.h(2)

# add a measurement to your circuit
qc.measure_all()

# visualize your circuit before running it
qc.draw("mpl")

## Qiskit Patterns Step 2: Optimize for target hardware

In [ ]:
# instantiate runtime service, choose backend

service = QiskitRuntimeService(channel="ibm_cloud")
backend = service.least_busy(simulator=False, operational=True)
backend.name

In [ ]:
# transpile circuit

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

# visualize transpiled circuit

isa_circuit.draw('mpl', idle_wires=False)

## Qiskit Patterns Step 3: Execute on target hardware

We are going to submit a number of shots which tells the system how many times to run our circuit. We could just run it once, but we'd have to trust that the system is in a superposition - we would only get one bitstring as a response. To prove to ourselves that our quantum system is being put into a superposition, and thus that we have an equal chance of getting any of the eight responses, we will run our circuit 1024 times, and plot all of the responses afterward.

In [ ]:
# run job for desired number of shots

sampler = Sampler(mode=backend)
job = sampler.run([isa_circuit], shots=1024)
print(job.job_id())

## Qiskit Patterns Step 4: Post-process results
On the [IBM Quantum platform](https://quantum.cloud.ibm.com/), you should click into either your most recent job (if this is your most recent) or the job matching the ID that was output above if you had multiple jobs running. There will be a piece of dynamic code within that workload that looks like the below but will autopopulate your API token and job ID when copy pasted.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    instance='instance'
)
job = service.job('JobID')
job_result = job.result()

# To get counts for a particular pub result, use
#
# pub_result = job_result[<idx>].data.<classical register>.get_counts()
#
# where <idx> is the index of the pub and <classical register> is the name of the classical register.
# You can use circuit.cregs to find the name of the classical registers.

In [ ]:
# get results in the form of counts

counts = job_result[0].data.meas.get_counts()
print(counts)

In [ ]:
# plot results

plot_histogram(counts)

We have modeled what it would look like to get 1024 separate responses from our Magic 8 Ball. We could just run it again, one time, to get a response for a single question (you are welcome to do so). For the sake of time, however, I will just take the first sampled measurement as our answer. The bitstrings in our counts item are added in the order that they were measured, so they are still random.

In [ ]:
# accessing the first key of the dict item containing our results
first_key = list(counts.keys())[0]

# turning that string into an integer
# result is given in base 2, so we need to communicate that because the int() function assumes base 10 as default
integer_value = int(first_key, 2)

# returns our Magic 8 Ball response
print(responses[integer_value])

### Expanding on the Magic 8 Ball

What if we could create a biased Magic 8 Ball, one that would increase the likelihood of measuring outcomes associated with positive or negative responses?

### Qiskit Patterns: Map problem to quantum circuits

In [ ]:
# set up a Quantum circuit with 3 qubits
qc_weighted = QuantumCircuit(3)

# place a Hadamard gate on qubits 0, 1, and 2
qc_weighted.h(0)
qc_weighted.h(1)
qc_weighted.h(2)

# weight the likelihood of certain outcomes by using an Ry gate
qc_weighted.ry(math.pi / 8, 0)

# add a measurement to your circuit
qc_weighted.measure_all()

# visualize the circuit before running it
qc_weighted.draw("mpl")

### Qiskit Patterns: Optimize for target hardware

In [ ]:
# instantiate runtime service, choose backend

service = QiskitRuntimeService(channel="ibm_cloud")
backend = service.least_busy(simulator=False, operational=True)
backend.name

In [ ]:
# transpile circuit

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc_weighted)

# visualize transpiled circuit

isa_circuit.draw('mpl', idle_wires=False)

### Qiskit Patterns: Execute on target hardware

In [ ]:
# run job for desired number of shots

sampler = Sampler(mode=backend)
job = sampler.run([isa_circuit], shots=1024)
print(job.job_id())

### Qiskit Patterns: Post-process results

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    instance='instance'
)
job = service.job('JobID')
job_result = job.result()

# To get counts for a particular pub result, use
#
# pub_result = job_result[<idx>].data.<classical register>.get_counts()
#
# where <idx> is the index of the pub and <classical register> is the name of the classical register.
# You can use circuit.cregs to find the name of the classical registers.

In [ ]:
# get results in the form of counts

counts = job_result[0].data.meas.get_counts()
print(counts)

In [ ]:
# plot results

plot_histogram(counts)

In [ ]:
# accessing the first key of the dict item containing our results
first_key = list(counts.keys())[0]

# turning that string into an integer
# result is given in base 2, so we need to communicate that because the int() function assumes base 10 as default
integer_value = int(first_key, 2)

# returns our Magic 8 Ball response
print(responses[integer_value])